In [ ]:
import sys
from sys import exit
import glob
import numpy as np
import pandas as pd
from os.path import join, isfile
from datetime import date, datetime
from unidecode import unidecode

from xlsxwriter.worksheet import (
    Worksheet, cell_number_tuple, cell_string_tuple)
    
from time import time, sleep
from contextlib import contextmanager
import functools
import threading


from my_funcs import ftime_elapsed, normalize_number, normalize_text, normalize_hash, trim_overspace, isvaliddate, normalize_cpf, normalize_date, date_hash, get_better, get_better_2, fit_cols, compare_dates, get_casos_confirmados, get_vacinados, get_tb_vacinados, get_srag

pd.set_option('display.max_columns', 0)
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

meses = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']
municipios = pd.read_csv('municipios.csv',converters={'ibge':str})
regionais = pd.read_csv('regionais.csv',converters={'ibge':str})

# faixa_etaria = [0,30,40,50,60,70,80,90,100]
# faixa_etaria_labels = ['err','0-29','30-39','40-49','50-59','60-69','70-79','80-89','90-99','>=100']
faixa_etaria = [60,65,70,75,80,85,90]
faixa_etaria_labels = ['0-59','60-64','65-69','70-74','75-79','80-84','85-89','>=90']
# faixa_etaria = [60,70,80,90]
# faixa_etaria_labels = ['0-59','60-69','70-79','80-89','>=90']

In [ ]:
init_time = time()
vacinados = get_vacinados(load=False)
print(ftime_elapsed(init_time), 'vacinados shape:', vacinados.shape)
vacinados['mes_aplicacao'] = vacinados['data_aplicacao'].apply(lambda x: meses[x.month-1] if isinstance(x,date) else 'erro')
vacinados.groupby('mes_aplicacao')[['sexo']].count()

In [ ]:
vacinados.columns

In [ ]:
vacinados.groupby('dose')[['sexo']].count()

In [ ]:
doses_header = ['cns','cpf','paciente','sexo','nome_mae','data_nascimento','ibge','data_aplicacao','fabricante','vacina_nome','hash_mae','hash_nasc'] #
primeira_dose = vacinados.loc[(vacinados['dose']=='1ª Dose')|(vacinados['dose']=='Única'),doses_header]
segunda_dose = vacinados.loc[vacinados['dose']=='2ª Dose',doses_header]

tb_vacinados = pd.merge(primeira_dose,segunda_dose, how='outer',on='cns',suffixes=['_1a_dose','_2a_dose'])

tb_vacinados['cpf'] = tb_vacinados.apply(lambda x: get_better(x['cpf_1a_dose'],x['cpf_2a_dose']), axis=1)
tb_vacinados['data_nascimento'] = tb_vacinados.apply(lambda x: get_better(x['data_nascimento_1a_dose'],x['data_nascimento_2a_dose'],datetime), axis=1)
tb_vacinados['nome_mae'] = tb_vacinados.apply(lambda x: get_better(x['nome_mae_1a_dose'],x['nome_mae_2a_dose']), axis=1)
tb_vacinados['paciente'] = tb_vacinados.apply(lambda x: get_better(x['paciente_1a_dose'],x['paciente_2a_dose']), axis=1)
tb_vacinados['fabricante'] = tb_vacinados.apply(lambda x: get_better(x['fabricante_1a_dose'],x['fabricante_2a_dose']), axis=1)
tb_vacinados['vacina_nome'] = tb_vacinados.apply(lambda x: get_better_2(x['vacina_nome_1a_dose'],x['vacina_nome_2a_dose']), axis=1)
tb_vacinados['hash_mae'] = tb_vacinados.apply(lambda x: get_better(x['hash_mae_1a_dose'],x['hash_mae_2a_dose']), axis=1)
tb_vacinados['hash_nasc'] = tb_vacinados.apply(lambda x: get_better(x['hash_nasc_1a_dose'],x['hash_nasc_2a_dose']), axis=1)
tb_vacinados['ibge'] = tb_vacinados.apply(lambda x: get_better(x['ibge_1a_dose'],x['ibge_2a_dose']), axis=1)
tb_vacinados['sexo'] = tb_vacinados.apply(lambda x: get_better(x['sexo_1a_dose'],x['sexo_2a_dose']), axis=1)

tb_vacinados = tb_vacinados.drop(columns=['cpf_1a_dose','cpf_2a_dose', 'data_nascimento_1a_dose', 'data_nascimento_2a_dose', 'paciente_1a_dose', 'paciente_2a_dose', 'fabricante_1a_dose', 'fabricante_2a_dose', 'vacina_nome_1a_dose', 'vacina_nome_2a_dose', 'nome_mae_1a_dose', 'nome_mae_2a_dose', 'hash_mae_1a_dose', 'hash_mae_2a_dose', 'hash_nasc_1a_dose', 'hash_nasc_2a_dose','ibge_1a_dose','ibge_2a_dose','sexo_2a_dose','sexo_1a_dose'])

tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'dias_entre_doses'] = (
    tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'data_aplicacao_2a_dose'] -
    tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].notnull()), 'data_aplicacao_1a_dose'])

tb_vacinados.loc[tb_vacinados['dias_entre_doses'].notnull(),'dias_entre_doses'] = tb_vacinados.loc[tb_vacinados['dias_entre_doses'].notnull(),'dias_entre_doses'].apply(lambda x: int(x.days))

tb_vacinados.loc[tb_vacinados['dias_entre_doses'].isnull(),'dias_entre_doses'] = 0

tb_vacinados['situacao'] = 'inconsistencia'

tb_vacinados.loc[(tb_vacinados['data_aplicacao_1a_dose'].notnull()) & (tb_vacinados['data_aplicacao_2a_dose'].isnull()), 'situacao'] = 'só primeira dose'

tb_vacinados.loc[(tb_vacinados['dias_entre_doses']>0), 'situacao'] = 'ambas doses aplicadas'

tb_vacinados['data_ultima_dose'] = tb_vacinados['data_aplicacao_1a_dose'].copy()
tb_vacinados.loc[tb_vacinados['data_aplicacao_2a_dose'].notna(),'data_ultima_dose'] = tb_vacinados.loc[tb_vacinados['data_aplicacao_2a_dose'].notna(),'data_aplicacao_2a_dose'].copy()

tb_vacinados['dias_apos_ultima_dose'] = tb_vacinados.apply(lambda x: (pd.to_datetime(date.today()) - x['data_ultima_dose']).days, axis=1)

vacinados['ibge'] = vacinados['ibge'].fillna('999999').apply(str)
regionais['ibge'] = regionais['ibge'].apply(str)

tb_vacinados = pd.merge(tb_vacinados,regionais[['ibge','nu_reg','nm_reg','nm_macro']],on='ibge',how='left')

tb_vacinados.loc[(tb_vacinados['data_nascimento'].notnull()) & (tb_vacinados['data_ultima_dose'].notnull()), 'idade'] = \
    tb_vacinados.loc[(tb_vacinados['data_nascimento'].notnull()) & (tb_vacinados['data_ultima_dose'].notnull())].apply(
            lambda row: row['data_ultima_dose'].year - row['data_nascimento'].year - (
                    (row['data_ultima_dose'].month, row['data_ultima_dose'].day) <
                    (row['data_nascimento'].month, row['data_nascimento'].day)
            ), axis=1
    )
tb_vacinados.loc[tb_vacinados['data_nascimento'].isna(), 'idade'] = -99
tb_vacinados['idade'] = tb_vacinados['idade'].apply(int)

tb_vacinados['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(tb_vacinados['idade'],faixa_etaria,right=False)]

print(len(tb_vacinados))
 
print(len(tb_vacinados.loc[tb_vacinados.duplicated("cns",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("cns", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("cpf",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("cpf", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("hash_mae",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("hash_mae", keep="first")

print(len(tb_vacinados.loc[tb_vacinados.duplicated("hash_nasc",keep="first")]))
tb_vacinados = tb_vacinados.drop_duplicates("hash_nasc", keep="first")

print(len(tb_vacinados))

tb_vacinados.groupby('faixa_etaria')[['cns']].count()

In [ ]:
bins = [1,2,3,7,14,21,30,60,90]
bins_label = ['hoje','24 horas','48 horas', '72 horas', '7 dias', '14 dias', '21 dias', '30 dias', '60 dias', '90 dias']
tb_vacinados['periodo_ultima_dose'] = [ x for x in np.digitize(tb_vacinados['dias_apos_ultima_dose'],bins,right=False)]
dias_apos_ultima_dose = tb_vacinados.groupby(['situacao','periodo_ultima_dose'])[['cns']].count().unstack().fillna(0).astype(int).rename(columns={'cns':'qtde'}).droplevel(0,1)
dias_apos_ultima_dose['total'] = dias_apos_ultima_dose.sum(1)
dias_apos_ultima_dose = dias_apos_ultima_dose.append(pd.DataFrame(dias_apos_ultima_dose.sum().tolist(),index=dias_apos_ultima_dose.columns.tolist(),columns=['total']).T)
dias_apos_ultima_dose.columns = [ bins_label[bins] if isinstance(bins,int) else bins for bins in dias_apos_ultima_dose.columns ]
dias_apos_ultima_dose = dias_apos_ultima_dose.reset_index()
dias_apos_ultima_dose.index = [0,2,1,3]
dias_apos_ultima_dose = dias_apos_ultima_dose.sort_index().set_index('index')
dias_apos_ultima_dose.index.name = None
dias_apos_ultima_dose

In [ ]:
tb_vacinados['periodo_ultima_dose'] = tb_vacinados['periodo_ultima_dose'].apply(lambda x: bins_label[x]).copy()
tb_vacinados = tb_vacinados[['ibge','nu_reg','nm_reg','nm_macro','cns', 'cpf', 'paciente', 'sexo', 'data_nascimento', 'idade', 'faixa_etaria', 'nome_mae', 'vacina_nome', 'fabricante', 'data_aplicacao_1a_dose', 'data_aplicacao_2a_dose', 'dias_entre_doses', 'situacao', 'dias_apos_ultima_dose', 'periodo_ultima_dose', 'data_ultima_dose', 'hash_mae', 'hash_nasc']]

In [ ]:
tb_vacinados = tb_vacinados.reset_index()
tb_vacinados.to_feather('tb_vacinados.feather')

In [ ]:
tb_vacinados = pd.read_feather('tb_vacinados.feather')
tb_vacinados = tb_vacinados.set_index('index')

In [ ]:
casos_confirmados = get_casos_confirmados(load=False)
casos_confirmados['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(casos_confirmados['idade'],faixa_etaria,right=False)]
casos_confirmados.groupby('evolucao')[['id']].count()

In [ ]:
print('casos_confirmados shape:', casos_confirmados.shape)
casos_confirmados.groupby('faixa_etaria')[['id']].count()

In [ ]:
casos_confirmados['comorbidade'] = casos_confirmados.apply(lambda x: 1 if any([
    x['doenca_cardiovascular'] == 1,
    x['hipertensao'] == 1,
    x['diabetes'] == 1,
    x['doenca_hepatica'] == 1,
    x['doenca_neurologica'] == 1,
    x['sindrome_down'] == 1,
    x['imunodeficiencia'] == 1,
    x['infeccao_hiv'] == 1,
    x['doenca_renal'] == 1,
    x['doenca_pulmonar'] == 1,
    x['neoplasia'] == 1,
    x['puerperio'] == 1,
    x['obesidade'] == 1,
    x['tabagismo'] == 1
]) else 0, axis=1)
casos_confirmados.groupby('comorbidade')[['id']].count()

In [ ]:
casos_confirmados.groupby('data_diagnostico')[['id']].count()

In [ ]:
casos_confirmados.loc[casos_confirmados['data_cura_obito'].notna()]

In [ ]:
# casos_confirmados.groupby('ocupacao')[['id']].count()

In [ ]:
# casos_confirmados['prof_saude'] = 'OUTROS TRABALHADORES'
# casos_confirmados.loc[casos_confirmados['ocupacao'].isin([1,3]),'prof_saude'] = 'PROFISSIONAL DA SAUDE'
# casos_confirmados.groupby('prof_saude')[['id']].count()

In [ ]:
obitos_confirmados = casos_confirmados.loc[(casos_confirmados['evolucao']=='OBITO')]
print('obitos_confirmados shape:', obitos_confirmados.shape)
obitos_confirmados.groupby('faixa_etaria')[['id']].count()

In [ ]:
casos_vacinados = casos_confirmados.loc[
    (casos_confirmados['cns'].notnull() & casos_confirmados['cns'].isin(tb_vacinados['cns'])) |
    (casos_confirmados['cpf'].notnull() & casos_confirmados['cpf'].isin(tb_vacinados['cpf'])) |
    (casos_confirmados['hash_mae'].notnull() & casos_confirmados['hash_mae'].isin(tb_vacinados['hash_mae'])) |
    (casos_confirmados['hash_nasc'].notnull() & casos_confirmados['hash_nasc'].isin(tb_vacinados['hash_nasc']))
]

print(len(casos_vacinados))

In [ ]:
tb_vacinados_cols = ['situacao','data_ultima_dose','vacina_nome','fabricante','data_aplicacao_1a_dose','data_aplicacao_2a_dose']

casos_vacinados = pd.merge(casos_confirmados.loc[casos_confirmados['cns'].notna()],tb_vacinados.loc[tb_vacinados['cns'].notna(),['cns']+tb_vacinados_cols], on='cns', how='inner')
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cns'].isin(casos_vacinados['cns'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['cpf'].notna()],tb_vacinados.loc[tb_vacinados['cpf'].notna(),['cpf']+tb_vacinados_cols], on='cpf', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cpf'].isin(casos_vacinados['cpf'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['hash_mae'].notna()],tb_vacinados.loc[tb_vacinados['hash_mae'].notna(),['hash_mae']+tb_vacinados_cols], on='hash_mae', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_mae'].isin(casos_vacinados['hash_mae'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

casos_vacinados = casos_vacinados.append(pd.merge(casos_confirmados.loc[casos_confirmados['hash_nasc'].notna()],tb_vacinados.loc[tb_vacinados['hash_nasc'].notna(),['hash_nasc']+tb_vacinados_cols], on='hash_nasc', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_nasc'].isin(casos_vacinados['hash_nasc'])].index)
print(len(casos_vacinados))
print(len(tb_vacinados))

print(len(casos_vacinados.loc[casos_vacinados.duplicated('id',keep=False)]))
casos_vacinados = casos_vacinados.drop_duplicates('id',keep='first')
len(casos_vacinados)

In [ ]:
casos_vacinados['diferenca_vacina_diag'] = casos_vacinados['data_diagnostico'] - casos_vacinados['data_ultima_dose']
casos_vacinados['diferenca_vacina_diag'] = casos_vacinados['diferenca_vacina_diag'].apply(lambda x: x.days)
casos_vacinados['situacao'].unique()
# casos_vacinados.groupby('vacina_nome')[['id']].count()

In [ ]:
casos_confirmados = casos_confirmados.loc[casos_confirmados['data_diagnostico'].notna()]
casos_confirmados.loc[casos_confirmados['data_recebimento'].notnull(), 'data_diagnostico'] = casos_confirmados.loc[casos_confirmados['data_recebimento'].notnull(), 'data_recebimento']
casos_confirmados.loc[casos_confirmados['data_coleta'].notnull(), 'data_diagnostico'] = casos_confirmados.loc[casos_confirmados['data_coleta'].notnull(), 'data_coleta']

In [ ]:
casos = pd.merge(casos_confirmados,casos_vacinados[['id','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='id')
casos['vacinado'] = 0 # sem vacina
casos.loc[(casos['diferenca_vacina_diag']>=0)&(casos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
casos.loc[(casos['situacao']=='ambas doses aplicadas')&(casos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado
casos['ano_caso'] = casos['data_diagnostico'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos['mes_caso'] = casos['data_diagnostico'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = casos.groupby(['ano_caso','mes_caso','faixa_etaria','vacinado'])[['id']].count().unstack(-2)
grupo = grupo.droplevel(0,1)
grupo = grupo.fillna(0).astype(float)
grupo.to_excel('casos_vacinados.xlsx')
grupo

In [ ]:
obitos_vacinados = casos_vacinados.loc[casos_vacinados['evolucao']=='OBITO'].copy()
obitos_vacinados['diferenca_vacina_obito'] = obitos_vacinados['data_cura_obito'] - obitos_vacinados['data_ultima_dose']
obitos_vacinados['diferenca_vacina_obito'] = obitos_vacinados['diferenca_vacina_obito'].apply(lambda x: x.days)

In [ ]:
obitos = pd.merge(obitos_confirmados,obitos_vacinados[['id','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='id')
obitos['vacinado'] = 0 # sem vacina
obitos.loc[(obitos['diferenca_vacina_diag']>=0)&(obitos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
obitos.loc[(obitos['situacao']=='ambas doses aplicadas')&(obitos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado

obitos['ano_obito'] = obitos['data_cura_obito'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
obitos['mes_obito'] = obitos['data_cura_obito'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = obitos.groupby(['ano_obito','mes_obito','faixa_etaria','vacinado'])[['id']].count().unstack(-2)
grupo = grupo.droplevel(0,1)

grupo = grupo.fillna(0).astype(float)
    
grupo.to_excel('obitos_vacinados.xlsx')
grupo

In [ ]:
obitos = pd.merge(obitos_confirmados,obitos_vacinados[['id','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='id')
obitos['vacinado'] = 0 # sem vacina
obitos.loc[(obitos['diferenca_vacina_diag']>=0)&(obitos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
obitos.loc[(obitos['situacao']=='ambas doses aplicadas')&(obitos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado

obitos = obitos.loc[(obitos['data_cura_obito'] > '2020-06-30') & (obitos['data_cura_obito'] < '2021-05-01' ) ]

grupo = obitos.groupby(['faixa_etaria','vacinado','comorbidade'])[['id']].count().unstack(-2).unstack(-1)

grupo = grupo.fillna(0).astype(int)
    
grupo.to_excel('comorbidade_raw.xlsx')
grupo

In [ ]:
tb_vacinados = pd.read_feather('tb_vacinados.feather')
tb_vacinados = tb_vacinados.set_index('index')

srag = get_srag(load=True)
srag = srag.loc[(srag['classi_fin']=='5')&(srag['sg_uf']=='PR')]

In [ ]:
srag['nu_idade_n'] = srag['nu_idade_n'].apply(int)
srag['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(srag['nu_idade_n'],faixa_etaria,right=False)]
obitos_srag = srag.loc[srag['evolucao']=='2']

In [ ]:
srag_vacinados = srag.loc[
    (srag['nu_cns'].notnull() & srag['nu_cns'].isin(tb_vacinados['cns'])) |
    (srag['nu_cpf'].notnull() & srag['nu_cpf'].isin(tb_vacinados['cpf'])) |
    (srag['hash_mae'].notnull() & srag['hash_mae'].isin(tb_vacinados['hash_mae'])) |
    (srag['hash_nasc'].notnull() & srag['hash_nasc'].isin(tb_vacinados['hash_nasc']))
]

print(len(srag_vacinados))
tb_vacinados.columns

In [ ]:
tb_vacinados_cols = ['paciente','nome_mae','data_nascimento','situacao','data_ultima_dose','vacina_nome','fabricante','data_aplicacao_1a_dose','data_aplicacao_2a_dose']
# tb_vacinados[tb_vacinados_cols]
srag_vacinados = pd.merge(srag.loc[srag['nu_cns'].notna()].rename(columns={'nu_cns':'cns'}),tb_vacinados.loc[tb_vacinados['cns'].notna(),['cns']+tb_vacinados_cols], on='cns', how='inner')
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cns'].isin(srag_vacinados['cns'])].index)
print(len(srag_vacinados))
print(len(tb_vacinados))

srag_vacinados = srag_vacinados.append(pd.merge(srag.loc[srag['nu_cpf'].notna()].rename(columns={'nu_cpf':'cpf'}),tb_vacinados.loc[tb_vacinados['cpf'].notna(),['cpf']+tb_vacinados_cols], on='cpf', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['cpf'].isin(srag_vacinados['cpf'])].index)
print(len(srag_vacinados))
print(len(tb_vacinados))

srag_vacinados = srag_vacinados.append(pd.merge(srag.loc[srag['hash_mae'].notna()],tb_vacinados.loc[tb_vacinados['hash_mae'].notna(),['hash_mae']+tb_vacinados_cols], on='hash_mae', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_mae'].isin(srag_vacinados['hash_mae'])].index)
print(len(srag_vacinados))
print(len(tb_vacinados))

srag_vacinados = srag_vacinados.append(pd.merge(srag.loc[srag['hash_nasc'].notna()],tb_vacinados.loc[tb_vacinados['hash_nasc'].notna(),['hash_nasc']+tb_vacinados_cols], on='hash_nasc', how='inner'), ignore_index=True)
tb_vacinados = tb_vacinados.drop(index=tb_vacinados.loc[tb_vacinados['hash_nasc'].isin(srag_vacinados['hash_nasc'])].index)
print(len(srag_vacinados))
print(len(tb_vacinados))

print(len(srag_vacinados.loc[srag_vacinados.duplicated('nu_notific',keep=False)]))
srag_vacinados = srag_vacinados.drop_duplicates('nu_notific',keep='first')
len(srag_vacinados)

In [ ]:
srag_vacinados.loc[(
#     (srag_vacinados['nu_cns'] != srag_vacinados['cns']) &
#     (srag_vacinados['nu_cpf'] != srag_vacinados['cpf']) &
    (srag_vacinados['nm_pacient'] != srag_vacinados['paciente']) 
#     (srag_vacinados['dt_nasc'] != srag_vacinados['data_nascimento']) &
#     (srag_vacinados['nm_mae_pac'] != srag_vacinados['nome_mae'])
),['nu_cns','cns','nu_cpf','cpf','nm_pacient','paciente','dt_nasc','data_nascimento','nm_mae_pac','nome_mae']]

In [ ]:
srag_vacinados.groupby('vacina_nome')[['nu_notific']].count()

In [ ]:
srag_vacinados.groupby('evolucao')[['nu_notific']].count()

In [ ]:
srag_vacinados.loc[srag_vacinados['evolucao']=='2']

In [ ]:
srag_vacinados['diferenca_vacina_diag'] = srag_vacinados['dt_pcr'] - srag_vacinados['data_ultima_dose']
srag_vacinados['diferenca_vacina_diag'] = srag_vacinados['diferenca_vacina_diag'].apply(lambda x: x.days)

casos = pd.merge(srag,srag_vacinados[['nu_notific','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='nu_notific')
casos['vacinado'] = 0 # sem vacina
casos.loc[(casos['diferenca_vacina_diag']>=0)&(casos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
casos.loc[(casos['situacao']=='ambas doses aplicadas')&(casos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado


casos['ano_caso'] = casos['dt_pcr'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos['mes_caso'] = casos['dt_pcr'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = casos.groupby(['ano_caso','mes_caso','faixa_etaria','vacinado'])[['nu_notific']].count().unstack(-2)
grupo = grupo.droplevel(0,1)

grupo = grupo.fillna(0).astype(float)

grupo.to_excel('casos_srag_vacinados.xlsx')
grupo

In [ ]:
obitos_srag_vacinados = srag_vacinados.loc[srag_vacinados['evolucao']=='2'].copy()
obitos_srag_vacinados['diferenca_vacina_obito'] = obitos_srag_vacinados['dt_evoluca'] - obitos_srag_vacinados['data_ultima_dose']
obitos_srag_vacinados['diferenca_vacina_obito'] = obitos_srag_vacinados['diferenca_vacina_obito'].apply(lambda x: x.days)

obitos = pd.merge(obitos_srag,obitos_srag_vacinados[['nu_notific','diferenca_vacina_diag']+tb_vacinados_cols],how='left',on='nu_notific')
obitos['vacinado'] = 0 # sem vacina
obitos.loc[(obitos['diferenca_vacina_diag']>=0)&(obitos['diferenca_vacina_diag']<=14),'vacinado'] = 1 #com vacina
obitos.loc[(obitos['situacao']=='ambas doses aplicadas')&(obitos['diferenca_vacina_diag']>14),'vacinado'] = 2 #imunizado

obitos['ano_obito'] = obitos['dt_evoluca'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
obitos['mes_obito'] = obitos['dt_evoluca'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
grupo = obitos.groupby(['ano_obito','mes_obito','faixa_etaria','vacinado'])[['nu_notific']].count().unstack(-2)
grupo = grupo.droplevel(0,1)

grupo = grupo.fillna(0).astype(float)
    
grupo.to_excel('obitos_srag_vacinados.xlsx')
grupo

In [ ]:
obitos_srag_vacinados.to_excel('obitos_srag_vacinados_data.xlsx')

In [ ]:
len(obitos)

In [ ]:
casos.loc[casos['vacina_nome'].isna(),'vacina_nome'] = 'SEM VACINA'
casos.groupby(['vacinado','vacina_nome'])[['nu_notific']].count().unstack(0).fillna(0)

In [ ]:
obitos.groupby(['ano_obito'])[['nu_notific']].count().unstack(0).fillna(0)